# Loop over satellite datafiles, making annotated images 

Annotation code from Claude, Sep 17 2025 
https://claude.ai/public/artifacts/2b14fb0a-0ee5-4590-ae5b-a9dc841e5970

Prompt 
You got it, I want the semi-transparent annotations to spread with time, conserving ink as if an amount had been added with an eyedropper at the event locations. Perhaps make room for a fading time scale,  beyond that spatial spreading effect, in case the accumulation gets too thick, although perhaps it will not since the hypothetical circles over which the 'ink' is spread can expand beyond the image area. I want three annotation layers. Two are similar, let's call them brown and green. Each is an expanding and widening circle, perhaps smoothed a bit at its perimeter, concentric on the event location where it was launched. The brown annulus moves 52 pixels per time step, the green one moves 23 pixels per time step. The third annotation is a little more complicated: for every event, select randomly from a list of dirtycolors = ['orange','red','pink'] and speeds = [10,7,4] pixels per time step. Unlike the brown and green annotations, which fade only in an ink-conserving way, and accumulate over time, these dirty circles erase all the other dirty colors and replace them with the new dirty color.


In [1]:
import numpy as np
import random
from collections import deque
import matplotlib.patches as patches
import cartopy.crs as ccrs

In [2]:
class EyedropperAnnotator:
    def __init__(self):
        # Store active events: [(lon, lat, birth_time), ...] for brown/green
        # [(lon, lat, birth_time, color, speed), ...] for dirty
        self.brown_events = deque()
        self.green_events = deque()
        self.dirty_events = deque()
        self.dirtycolors = ['orange', 'red', 'pink']
        self.speeds = [10, 7, 4]
        
    def add_events(self, event_locations, current_time):
        """Add new events at current time"""
        for lon, lat in event_locations:
            # Add brown and green events
            self.brown_events.append((lon, lat, current_time))
            self.green_events.append((lon, lat, current_time))
            
            # Add random dirty event (clears previous dirty events)
            color = random.choice(self.dirtycolors)
            speed = random.choice(self.speeds)
            self.dirty_events.clear()  # Erase previous dirty colors
            self.dirty_events.append((lon, lat, current_time, color, speed))
    
    def _draw_ring(self, overlay, x, y, inner_r, outer_r, color, alpha):
        """Draw a smoothed ring with given transparency"""
        if outer_r <= 0:
            return
            
        # Create mask for the ring
        mask = np.zeros((self.h, self.w), dtype=np.uint8)
        if outer_r > 0:
            cv2.circle(mask, (int(x), int(y)), int(outer_r), 255, -1)
        if inner_r > 0:
            cv2.circle(mask, (int(x), int(y)), int(inner_r), 0, -1)
        
        # Smooth the edges
        mask = cv2.GaussianBlur(mask, (5, 5), 1.0)
        
        # Apply color with alpha blending
        mask_norm = mask.astype(np.float32) / 255.0 * alpha
        for c in range(3):
            overlay[:, :, c] = overlay[:, :, c] * (1 - mask_norm) + color[c] * mask_norm
    
    def plot_annotations(self, ax, current_time):
        """Plot annotations directly on matplotlib axes"""
        import matplotlib.patches as patches
        
        # Brown annotations (52 degrees per time step)
        for lon, lat, birth_time in list(self.brown_events):
            age = current_time - birth_time
            if age < 0:
                continue
                
            outer_r = age * 52  # degrees
            inner_r = max(0, outer_r - 30)
            
            # Ink-conserving alpha
            area = np.pi * (outer_r**2 - inner_r**2) if outer_r > inner_r else np.pi
            alpha = min(0.8, 1000.0 / max(area, 1))
            
            if alpha > 0.01:
                # Draw ring as wedge minus inner circle
                if outer_r > 0:
                    circle = patches.Circle((lon, lat), outer_r, 
                                          facecolor='saddlebrown', alpha=alpha, 
                                          edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(circle)
                if inner_r > 0:
                    inner_circle = patches.Circle((lon, lat), inner_r, 
                                                facecolor='white', alpha=1.0,
                                                edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(inner_circle)
            else:
                self.brown_events.remove((lon, lat, birth_time))
        
        # Green annotations (23 degrees per time step)
        for lon, lat, birth_time in list(self.green_events):
            age = current_time - birth_time
            if age < 0:
                continue
                
            outer_r = age * 23
            inner_r = max(0, outer_r - 20)
            
            area = np.pi * (outer_r**2 - inner_r**2) if outer_r > inner_r else np.pi
            alpha = min(0.8, 800.0 / max(area, 1))
            
            if alpha > 0.01:
                if outer_r > 0:
                    circle = patches.Circle((lon, lat), outer_r,
                                          facecolor='green', alpha=alpha,
                                          edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(circle)
                if inner_r > 0:
                    inner_circle = patches.Circle((lon, lat), inner_r,
                                                facecolor='white', alpha=1.0,
                                                edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(inner_circle)
            else:
                self.green_events.remove((lon, lat, birth_time))
        
        # Dirty color annotations
        for lon, lat, birth_time, color, speed in list(self.dirty_events):
            age = current_time - birth_time
            if age < 0:
                continue
                
            outer_r = age * speed
            inner_r = max(0, outer_r - 15)
            
            area = np.pi * (outer_r**2 - inner_r**2) if outer_r > inner_r else np.pi
            alpha = min(0.7, 600.0 / max(area, 1))
            
            if alpha > 0.01:
                if outer_r > 0:
                    circle = patches.Circle((lon, lat), outer_r,
                                          facecolor=color, alpha=alpha,
                                          edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(circle)
                if inner_r > 0:
                    inner_circle = patches.Circle((lon, lat), inner_r,
                                                facecolor='white', alpha=1.0,
                                                edgecolor='none', transform=ccrs.PlateCarree())
                    ax.add_patch(inner_circle)
            else:
                self.dirty_events.remove((lon, lat, birth_time, color, speed))

In [3]:
# code to detect events (cells) in an image 
# event_locations = detect_events(data_array)  # Returns list of tuples [(lon1,lat1), (lon2,lat2), ...]

def detect_events(data_array):
    events = []

    events.append()
    
    return events 

In [4]:
# Define the folder path to the data files 
folder_path = '/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/Test10files/'
folder_path = '/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/'
output_path = '/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/Images/'

#output_pdf = '/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/Images.pdf'    # mkdir if needed

varname = 'irwin_cdr'

In [5]:
# loop over xarray files making .png images, maybe WITH ANNOTATIONS

import os
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Initialize annotator
# annotator = EyedropperAnnotator((600, 1000))  # Approximate figure size in pixels


# Loop over all files in the folder
filenames = sorted(os.listdir(folder_path))
for it,filename in enumerate(filenames):

    file_path = os.path.join(folder_path, filename)
    
    # Check if the file is a valid data file (e.g., NetCDF, GRIB, GeoTIFF)
    if filename.lower().endswith(('.nc', '.grib', '.tif', '.tiff')):
        print(file_path)

        # Open the file with xarray
        ds = xr.open_dataset(file_path)  # Adjust engine as needed
        
        # Select a variable and convert to grayscale (assuming it's a single variable)
        # Replace 'variable_name' with the actual variable name in your dataset
        data_array = ds[varname]  # Replace with actual variable name

        
# Central area only
        #xarray.DataArray'irwin_cdr'time: 1lat: 428lon: 572 
        data_array = data_array.isel(lat = slice(64,364), lon=slice(86,486))


        
     # Your existing code to detect events
#        event_locations = detect_events(data_array)  # Returns [(lon1,lat1), (lon2,lat2), ...]
        
     # Create your existing plot
        fig = plt.figure(figsize=(10, 6))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
        data_array.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='gray_r', vmin=200, vmax=300,
                        add_colorbar=False)
        ax.coastlines(resolution='10m', color='black')
     
    # Add eyedropper annotations
#        annotator.add_events(event_locations, it)
#        annotator.plot_annotations(ax, it)

        # Save the image to disk
        output_filename = output_path + filename + '.png'
        #output_filename = output_path + filename + '.annot.png'
        plt.savefig(output_filename, dpi=150, bbox_inches='tight')        
        plt.close(fig)   


/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time000.nc
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time001.nc
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time002.nc
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time003.nc
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time004.nc
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time005.nc
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time006.nc
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time007.nc
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time008.nc
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time009.nc
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time010.nc
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time011.nc
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time012.nc
/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/time013.nc
/Users/bmapes/Box/Sky_Symphony_Box

In [6]:
data_array

<xarray.DataArray 'irwin_cdr' (time: 1, lat: 300, lon: 400)> Size: 480kB
[120000 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 1kB 12.81 12.88 12.95 13.02 ... 33.6 33.67 33.74
  * lon      (lon) float32 2kB -121.6 -121.6 -121.5 ... -93.83 -93.76 -93.69
  * time     (time) datetime64[ns] 8B 2012-08-17T15:00:00
Attributes:
    long_name:          NOAA FCDR of Brightness Temperature near 11 microns (...
    standard_name:      toa_brightness_temperature
    units:              Kelvin
    comment:            Nadir-most observations
    valid_range:        [140. 375.]
    Note_A:             Calibration correction of Kanpp (2007) not applied
    Note_B:             The IRWIN_CDR data have been View Zenith corrected us...
    intergeo_cal_flag:  0
    _ChunkSizes:        [   1 2000 5143]

In [7]:
428/2-150, 428/2+150, 572/2-200, 572/2+200

(64.0, 364.0, 86.0, 486.0)

In [25]:
# Loop over xarry files making PDF file of images: slow (thanks Leo) 
fmansadk

import os
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

# Create a PDF file and loop through the images
with PdfPages(output_pdf) as pdf:

    # Loop over all files in the folder
    for filename in sorted(os.listdir(folder_path)):
        file_path = os.path.join(folder_path, filename)
        print(file_path)
        
        # Check if the file is a valid data file (e.g., NetCDF, GRIB, GeoTIFF)
        if filename.lower().endswith(('.nc', '.grib', '.tif', '.tiff')):
            # Open the file with xarray
            ds = xr.open_dataset(file_path)  # Adjust engine as needed
            
            # Select a variable and convert to grayscale (assuming it's a single variable)
            # Replace 'variable_name' with the actual variable name in your dataset
            data_array = ds[varname]  # Replace with actual variable name
            
            # Create a plot with Cartopy using PlateCarree projection
            fig = plt.figure(figsize=(10, 6))
            ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
            
            # Plot the data in reversed gray colorscale 
            data_array.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='gray_r', add_colorbar=False)
            
            # Add coastlines
            ax.coastlines(resolution='10m', color='black')
            
            # Set title
            plt.title(f"Grayscale Plot of {filename}")
            
            # Save the image to disk
            output_filename = output_path + filename + '.png'
            plt.savefig(output_filename, dpi=150, bbox_inches='tight')
            
            # Save the figure to the PDF
            pdf.savefig(fig, bbox_inches='tight')
            
            # Close the figure to free memory
            plt.close(fig)


NameError: name 'fmansadk' is not defined

In [ ]:
# OLD DELETEME loop over xarray files making images WITH ANNOTATIONS
gjsioathseij

import os
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Define the folder path to the data files 
folder_path = '/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/Test10files/'
output_path = '/Users/bmapes/Box/Sky_Symphony_Box/ERNESTO12/GridSatBoxes/Images/'    # mkdir if needed
varname = 'irwin_cdr'

# Loop over all files in the folder
filenames = sorted(os.listdir(folder_path))
for it,filename in enumerate(filenames):
    
    file_path = os.path.join(folder_path, filename)
    print(file_path)
    
    # Check if the file is a valid data file (e.g., NetCDF, GRIB, GeoTIFF)
    if filename.lower().endswith(('.nc', '.grib', '.tif', '.tiff')):
        # Open the file with xarray
        ds = xr.open_dataset(file_path)  # Adjust engine as needed
        
        # Select a variable and convert to grayscale (assuming it's a single variable)
        # Replace 'variable_name' with the actual variable name in your dataset
        data_array = ds[varname]  # Replace with actual variable name
        
        # Create a plot with Cartopy using PlateCarree projection
        fig = plt.figure(figsize=(10, 6))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
        
        # Plot the data in reversed gray colorscale 
        data_array.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='gray_r', add_colorbar=False)
        
        # Add coastlines
        ax.coastlines(resolution='10m', color='black')
        
        # Set title
        plt.title(f"Grayscale Plot of {filename}")

# Usage in your loop:
        annotator = EyedropperAnnotator(data_array.shape)
        event_locations = detect_events(image_array)  # Returns [(x1,y1), (x2,y2), ...]
        annotator.add_events(event_locations, t)
        annotated_image = annotator.annotate_image(image_array, t)
        cv2.imwrite(f'frame_{t:04d}.png', annotated_image)
        
        # Save the image to disk
        output_filename = output_path + filename + '.png'
        plt.savefig(output_filename, dpi=150, bbox_inches='tight')
        
        # Close the figure to free memory
        plt.close(fig)   